In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1rhbO-3WtvU0YjQYQrCebpE_IATSZyRCX", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 Tensor Parallelism: Splitting Layers Across GPUs

*Part 3 of the Vizuara series on 5D Parallelism from Scratch*
*Estimated time: 45 minutes*

In this notebook, you will tear a single weight matrix apart, distribute its pieces across simulated GPUs, and stitch the results back together — all from scratch. By the end, you will have a working Megatron-style tensor-parallel MLP and a tensor-parallel multi-head attention module.

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://course-creator-brown.vercel.app/courses/5d-parallelism-from-scratch/practice/3/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
# Setup and imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

# We simulate multi-GPU by splitting tensors on a single device
device = torch.device('cpu')
print(f"PyTorch version: {torch.__version__}")
print("We will simulate tensor parallelism on a single device.")
print("In production, each 'rank' would be a separate GPU.")

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

In the previous notebook, we saw that Data Parallelism gives each GPU a **full copy** of the model. That is fine when the model fits on one GPU. But what happens when a **single layer** is too large?

Consider a 70B parameter model like Llama 2 70B. Its MLP layers have weight matrices of size **(8192 x 28672)** — that is **235 million parameters** in a single matrix, or **470 MB** in fp16. And there are *two* such matrices per MLP block, plus the attention projections.

The total parameter memory for just the weights of a 70B model is about **140 GB** in fp16. A single A100 has 80 GB. Even with ZeRO Stage 3, the communication overhead of constantly fetching weight shards can be painful for operations that happen inside every single layer.

Tensor Parallelism offers an elegant alternative: **split the weight matrices themselves** across GPUs, so each GPU only stores and computes a fraction of each layer.

In [ ]:
# Let us see the concrete numbers
hidden_dim = 8192
intermediate_dim = 28672  # ~3.5x hidden, common in Llama-style models

params_per_mlp_weight = hidden_dim * intermediate_dim
bytes_fp16 = params_per_mlp_weight * 2  # 2 bytes per fp16 parameter

print(f"One MLP weight matrix: {hidden_dim} x {intermediate_dim}")
print(f"Parameters: {params_per_mlp_weight:,} ({params_per_mlp_weight/1e6:.0f}M)")
print(f"Memory (fp16): {bytes_fp16 / 1e6:.0f} MB")
print(f"\nWith 2-way Tensor Parallelism:")
print(f"  Each GPU stores: {hidden_dim} x {intermediate_dim // 2} = {params_per_mlp_weight // 2:,} params")
print(f"  Memory per GPU: {bytes_fp16 / 2 / 1e6:.0f} MB  (halved!)")
print(f"\nWith 8-way Tensor Parallelism (full NVLink node):")
print(f"  Each GPU stores: {hidden_dim} x {intermediate_dim // 8} = {params_per_mlp_weight // 8:,} params")
print(f"  Memory per GPU: {bytes_fp16 / 8 / 1e6:.0f} MB  (8x reduction!)")

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Let us go back to our restaurant analogy. In Data Parallelism, every chef had a complete copy of the recipe and worked on different orders. In Tensor Parallelism, we **tear one page of the recipe in half**.

Imagine Chef A and Chef B are making a complex sauce. The recipe says: "Combine 6 ingredients." Instead of each chef handling all 6, Chef A handles ingredients 1-3 and Chef B handles ingredients 4-6. They each do their mixing independently, and at the end, they combine their partial sauces into the final dish.

The key insight is this: **we are splitting the computation of a single matrix multiply across GPUs**. Not splitting data, not splitting layers — splitting the arithmetic inside one operation.

### 🤔 Think About This

If $Y = XW$, and the weight matrix $W$ is too large for one GPU, how could you split this computation?

There are two natural ways:

1. **Split $W$ by columns** (vertically) — each GPU gets some output features
2. **Split $W$ by rows** (horizontally) — each GPU gets part of the dot product

Think about what happens to the shapes in each case. What operation do you need at the end to reconstruct the full result?

*Take a moment to reason about this before scrolling down.*

In [ ]:
# Quick shape check: let us set up a small example to keep in mind
# as we work through the math below
X_example = torch.randn(2, 4)  # 2 samples, 4 features
W_example = torch.randn(4, 6)  # maps 4 inputs to 6 outputs
Y_example = X_example @ W_example
print(f"X: {X_example.shape}  @  W: {W_example.shape}  =  Y: {Y_example.shape}")
print("Our goal: compute this same Y, but split the work across 2 GPUs.")

In [ ]:
#@title 🎧 Listen: Math Column Split
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_math_column_split.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### Column-Wise Split

We split the weight matrix $W$ along its **columns** (the output dimension):

$$W = [W_1 \mid W_2]$$

The matrix multiply becomes:

$$Y = X \cdot [W_1 \mid W_2] = [X W_1 \mid X W_2]$$

Computationally, this means: GPU 0 computes $X W_1$ (a subset of output features), GPU 1 computes $X W_2$ (the rest of the output features), and we **concatenate** the results along the feature dimension.

No information is lost. No approximation is made. This is **mathematically exact**.

### Row-Wise Split

Alternatively, we split $W$ along its **rows** (the input dimension):

$$W = \begin{bmatrix} W_1 \\ W_2 \end{bmatrix}$$

But now we must also split the input $X$ along the corresponding dimension:

$$X = [X_1 \mid X_2]$$

The matrix multiply becomes:

$$Y = [X_1 \mid X_2] \cdot \begin{bmatrix} W_1 \\ W_2 \end{bmatrix} = X_1 W_1 + X_2 W_2$$

Computationally, this means: GPU 0 computes the partial product $X_1 W_1$, GPU 1 computes $X_2 W_2$, and we **sum** them via an AllReduce operation. Each partial product has the **same shape** as the final output — we are accumulating partial sums, not concatenating slices.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_math_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Let us verify the math identities with our small example
W1_col = W_example[:, :3]   # Column split: first 3 output features
W2_col = W_example[:, 3:]   # Column split: last 3 output features
Y_col = torch.cat([X_example @ W1_col, X_example @ W2_col], dim=1)

W1_row = W_example[:2, :]   # Row split: first 2 input features
W2_row = W_example[2:, :]   # Row split: last 2 input features
Y_row = X_example[:, :2] @ W1_row + X_example[:, 2:] @ W2_row

print(f"Column-wise matches: {torch.allclose(Y_col, Y_example)}")
print(f"Row-wise matches:    {torch.allclose(Y_row, Y_example)}")
print("Both are exact — zero error. These are mathematical identities.")

### 💡 Key Difference

| | Column-Wise | Row-Wise |
|---|---|---|
| **What is split** | Output features | Input features |
| **Input needed** | Full $X$ on each GPU | Split $X$ across GPUs |
| **Recombination** | Concatenate | AllReduce (sum) |
| **Output per GPU** | Partial output (fewer columns) | Full-shaped partial sum |

This distinction will matter enormously when we wire them together in the Megatron-LM MLP.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_numerical_example.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let Us Build It — Component by Component

### 4.1 Numerical Example: Seeing the Math with Real Numbers

Before writing any classes, let us plug in some simple numbers and verify the math by hand.

In [ ]:
# Create small, concrete matrices
# X: (2 x 4) — 2 samples, 4 input features
# W: (4 x 6) — maps 4 inputs to 6 outputs
X = torch.tensor([
    [1.0, 2.0, 3.0, 4.0],
    [5.0, 6.0, 7.0, 8.0]
])
W = torch.tensor([
    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
    [0.7, 0.8, 0.9, 1.0, 1.1, 1.2],
    [1.3, 1.4, 1.5, 1.6, 1.7, 1.8],
    [1.9, 2.0, 2.1, 2.2, 2.3, 2.4],
])

# Ground truth: full matmul
Y_full = X @ W
print("X shape:", X.shape)
print("W shape:", W.shape)
print("Y = X @ W shape:", Y_full.shape)
print("\nFull result Y:")
print(Y_full)

Now let us split this computation two ways and verify we get the exact same result.

In [ ]:
# === Column-Wise Split ===
# Split W along columns: W1 = first 3 cols, W2 = last 3 cols
W1_col = W[:, :3]  # shape (4, 3)
W2_col = W[:, 3:]  # shape (4, 3)

# Each "GPU" gets the full X and its shard of W
Y1_col = X @ W1_col  # GPU 0: shape (2, 3)
Y2_col = X @ W2_col  # GPU 1: shape (2, 3)

# Reconstruct by concatenation
Y_col = torch.cat([Y1_col, Y2_col], dim=1)  # shape (2, 6)

print("=== Column-Wise Split ===")
print(f"W1 shape: {W1_col.shape}, W2 shape: {W2_col.shape}")
print(f"GPU 0 output: {Y1_col.shape}, GPU 1 output: {Y2_col.shape}")
print(f"Concatenated: {Y_col.shape}")
print(f"Matches full result: {torch.allclose(Y_col, Y_full)}")
print(f"Max error: {(Y_col - Y_full).abs().max().item():.1e}")

Now the row-wise split, where we sum partial products instead of concatenating.

In [ ]:
# === Row-Wise Split ===
# Split W along rows: W1 = first 2 rows, W2 = last 2 rows
W1_row = W[:2, :]  # shape (2, 6)
W2_row = W[2:, :]  # shape (2, 6)

# Also split X along columns (matching dimension)
X1 = X[:, :2]  # shape (2, 2)
X2 = X[:, 2:]  # shape (2, 2)

# Each "GPU" computes a partial sum
Y1_row = X1 @ W1_row  # GPU 0: shape (2, 6)
Y2_row = X2 @ W2_row  # GPU 1: shape (2, 6)

# Reconstruct by summing (this is the AllReduce)
Y_row = Y1_row + Y2_row  # shape (2, 6)

print("=== Row-Wise Split ===")
print(f"W1 shape: {W1_row.shape}, W2 shape: {W2_row.shape}")
print(f"X1 shape: {X1.shape}, X2 shape: {X2.shape}")
print(f"GPU 0 partial sum: {Y1_row.shape}, GPU 1 partial sum: {Y2_row.shape}")
print(f"After AllReduce (sum): {Y_row.shape}")
print(f"Matches full result: {torch.allclose(Y_row, Y_full)}")
print(f"Max error: {(Y_row - Y_full).abs().max().item():.1e}")

Both methods produce **exactly** the same result as the unsplit computation. Zero error. This is not an approximation — it is a mathematical identity.

### 📊 Visualization: Column-Wise vs Row-Wise Splits

Let us visualize what is happening to the weight matrix in each case.

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_split_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize the full weight matrix and the two splitting strategies
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Full weight matrix
im0 = axes[0].imshow(W.numpy(), cmap='RdYlBu_r', aspect='auto')
axes[0].set_title('Full Weight Matrix W\n(4 x 6)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Output features')
axes[0].set_ylabel('Input features')
plt.colorbar(im0, ax=axes[0], shrink=0.8)

# Column-wise split
W_col_viz = W.numpy().copy()
axes[1].imshow(W_col_viz[:, :3], cmap='Blues', aspect='auto',
               extent=[-0.5, 2.5, 3.5, -0.5])
axes[1].imshow(W_col_viz[:, 3:], cmap='Greens', aspect='auto',
               extent=[3.5, 6.5, 3.5, -0.5], alpha=0.9)
axes[1].axvline(x=3.0, color='red', linewidth=3, linestyle='--')
axes[1].set_title('Column-Wise Split\nGPU 0 (blue) | GPU 1 (green)',
                   fontsize=12, fontweight='bold')
axes[1].set_xlabel('Output features')
axes[1].set_ylabel('Input features')

The left panel shows the full weight matrix. Now let us add the row-wise split panel.

In [ ]:
# Row-wise split (continuing the figure from the previous cell)
axes[2].imshow(W_col_viz[:2, :], cmap='Blues', aspect='auto',
               extent=[-0.5, 5.5, 1.5, -0.5])
axes[2].imshow(W_col_viz[2:, :], cmap='Greens', aspect='auto',
               extent=[-0.5, 5.5, 3.5, 1.5], alpha=0.9)
axes[2].axhline(y=1.5, color='red', linewidth=3, linestyle='--')
axes[2].set_title('Row-Wise Split\nGPU 0 (blue) / GPU 1 (green)',
                   fontsize=12, fontweight='bold')
axes[2].set_xlabel('Output features')
axes[2].set_ylabel('Input features')

plt.tight_layout()
plt.savefig('tensor_split_visualization.png', dpi=100, bbox_inches='tight')
plt.show()
print("The red dashed lines show where we cut the weight matrix.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_column_parallel_class.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Column-Wise Parallel Linear Layer

Now let us wrap the column-wise split into a reusable class. This simulates what would happen on a multi-GPU system — each "rank" (GPU index) computes its portion.

In [ ]:
class ColumnParallelLinear:
    """
    Splits a linear layer along the OUTPUT dimension (columns).

    Each GPU stores W[:, start:end] and computes X @ W_shard.
    The outputs are concatenated across GPUs to form the full result.
    """
    def __init__(self, weight, bias=None, world_size=2):
        self.world_size = world_size
        self.full_weight = weight  # (in_features, out_features)
        self.full_bias = bias

        # Split weight into column shards
        out_features = weight.shape[1]
        assert out_features % world_size == 0, \
            f"out_features ({out_features}) must be divisible by world_size ({world_size})"

        shard_size = out_features // world_size
        self.shards = [weight[:, i*shard_size:(i+1)*shard_size]
                       for i in range(world_size)]
        self.bias_shards = None
        if bias is not None:
            self.bias_shards = [bias[i*shard_size:(i+1)*shard_size]
                                for i in range(world_size)]

Each shard holds a vertical slice of the weight matrix. Now let us add the forward methods.

In [ ]:
def column_parallel_forward(col_linear, x, rank):
    """Compute output for a specific GPU rank."""
    out = x @ col_linear.shards[rank]
    if col_linear.bias_shards is not None:
        out = out + col_linear.bias_shards[rank]
    return out

def column_parallel_forward_all(col_linear, x):
    """Simulate all GPUs computing, then concatenating results."""
    outputs = [column_parallel_forward(col_linear, x, rank)
               for rank in range(col_linear.world_size)]
    return torch.cat(outputs, dim=-1)

# Attach methods to the class for convenient use
ColumnParallelLinear.forward = lambda self, x, rank: column_parallel_forward(self, x, rank)
ColumnParallelLinear.forward_all_and_gather = lambda self, x: column_parallel_forward_all(self, x)

Let us test that our column-parallel layer produces the same result as the full computation.

In [ ]:
# Test ColumnParallelLinear
torch.manual_seed(42)
in_features, out_features = 8, 12
W_test = torch.randn(in_features, out_features)
b_test = torch.randn(out_features)
X_test = torch.randn(4, in_features)  # batch of 4

col_linear = ColumnParallelLinear(W_test, b_test, world_size=2)

# Full computation (ground truth)
Y_expected = X_test @ W_test + b_test

# Tensor-parallel computation
Y_tp = col_linear.forward_all_and_gather(X_test)

print("ColumnParallelLinear Test")
print(f"  Input shape: {X_test.shape}")
print(f"  Full weight shape: {W_test.shape}")
print(f"  Shard 0 shape: {col_linear.shards[0].shape}")
print(f"  Shard 1 shape: {col_linear.shards[1].shape}")
print(f"  GPU 0 output shape: {col_linear.forward(X_test, 0).shape}")
print(f"  GPU 1 output shape: {col_linear.forward(X_test, 1).shape}")
print(f"  Gathered output shape: {Y_tp.shape}")
print(f"  Matches full computation: {torch.allclose(Y_tp, Y_expected, atol=1e-6)}")
print(f"  Max error: {(Y_tp - Y_expected).abs().max().item():.2e}")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_row_parallel_class.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Row-Wise Parallel Linear Layer

The row-wise split is the complement: we split the weight along the **input** dimension and sum the partial results.

In [ ]:
class RowParallelLinear:
    """
    Splits a linear layer along the INPUT dimension (rows).

    Each GPU stores W[start:end, :] and receives X[:, start:end].
    The outputs are summed across GPUs (AllReduce) to form the full result.
    """
    def __init__(self, weight, bias=None, world_size=2):
        self.world_size = world_size
        self.full_weight = weight  # (in_features, out_features)
        self.full_bias = bias

        # Split weight into row shards
        in_features = weight.shape[0]
        assert in_features % world_size == 0, \
            f"in_features ({in_features}) must be divisible by world_size ({world_size})"

        shard_size = in_features // world_size
        self.shards = [weight[i*shard_size:(i+1)*shard_size, :]
                       for i in range(world_size)]
        self.shard_size = shard_size

Now let us add the forward methods for the row-parallel layer. Notice how the AllReduce (sum) combines partial outputs.

In [ ]:
def row_parallel_forward(row_linear, x_shard, rank):
    """
    Compute partial output for a specific GPU rank.
    x_shard should be the rank's portion of the input features.
    """
    return x_shard @ row_linear.shards[rank]

def row_parallel_forward_all(row_linear, x):
    """Simulate all GPUs computing partial sums, then AllReduce."""
    partial_sums = []
    for rank in range(row_linear.world_size):
        start = rank * row_linear.shard_size
        end = (rank + 1) * row_linear.shard_size
        x_shard = x[:, start:end]
        partial_sums.append(row_parallel_forward(row_linear, x_shard, rank))

    # AllReduce: sum all partial outputs
    result = sum(partial_sums)
    if row_linear.full_bias is not None:
        result = result + row_linear.full_bias
    return result

# Attach methods to the class
RowParallelLinear.forward = lambda self, x, rank: row_parallel_forward(self, x, rank)
RowParallelLinear.forward_all_and_reduce = lambda self, x: row_parallel_forward_all(self, x)

Let us verify the row-parallel layer matches the full computation.

In [ ]:
# Test RowParallelLinear
row_linear = RowParallelLinear(W_test, b_test, world_size=2)

# Full computation (ground truth)
Y_expected = X_test @ W_test + b_test

# Tensor-parallel computation
Y_tp_row = row_linear.forward_all_and_reduce(X_test)

print("RowParallelLinear Test")
print(f"  Input shape: {X_test.shape}")
print(f"  Full weight shape: {W_test.shape}")
print(f"  Shard 0 shape: {row_linear.shards[0].shape}")
print(f"  Shard 1 shape: {row_linear.shards[1].shape}")
print(f"  Reduced output shape: {Y_tp_row.shape}")
print(f"  Matches full computation: {torch.allclose(Y_tp_row, Y_expected, atol=1e-6)}")
print(f"  Max error: {(Y_tp_row - Y_expected).abs().max().item():.2e}")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_shards_visualization.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Weight Shards Side by Side

Let us look at the actual weight shards for both split strategies.

In [ ]:
# Top row: Column-wise split
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

im00 = axes[0, 0].imshow(W_test.numpy(), cmap='RdYlBu_r', aspect='auto')
axes[0, 0].set_title('Full W (8x12)', fontweight='bold')
axes[0, 0].set_ylabel('Column Split', fontsize=12, fontweight='bold')
plt.colorbar(im00, ax=axes[0, 0], shrink=0.7)

im01 = axes[0, 1].imshow(col_linear.shards[0].numpy(), cmap='Blues', aspect='auto')
axes[0, 1].set_title('GPU 0: W[:, :6]\n(8x6)', fontweight='bold')
plt.colorbar(im01, ax=axes[0, 1], shrink=0.7)

im02 = axes[0, 2].imshow(col_linear.shards[1].numpy(), cmap='Greens', aspect='auto')
axes[0, 2].set_title('GPU 1: W[:, 6:]\n(8x6)', fontweight='bold')
plt.colorbar(im02, ax=axes[0, 2], shrink=0.7)

Now the bottom row showing the row-wise split.

In [ ]:
# Bottom row: Row-wise split
im10 = axes[1, 0].imshow(W_test.numpy(), cmap='RdYlBu_r', aspect='auto')
axes[1, 0].set_title('Full W (8x12)', fontweight='bold')
axes[1, 0].set_ylabel('Row Split', fontsize=12, fontweight='bold')
plt.colorbar(im10, ax=axes[1, 0], shrink=0.7)

im11 = axes[1, 1].imshow(row_linear.shards[0].numpy(), cmap='Blues', aspect='auto')
axes[1, 1].set_title('GPU 0: W[:4, :]\n(4x12)', fontweight='bold')
plt.colorbar(im11, ax=axes[1, 1], shrink=0.7)

im12 = axes[1, 2].imshow(row_linear.shards[1].numpy(), cmap='Greens', aspect='auto')
axes[1, 2].set_title('GPU 1: W[4:, :]\n(4x12)', fontweight='bold')
plt.colorbar(im12, ax=axes[1, 2], shrink=0.7)

plt.suptitle('Column-Wise vs Row-Wise Weight Sharding',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('weight_shards.png', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
#@title 🎧 Listen: Megatron Mlp Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_megatron_mlp_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 The Megatron-LM MLP Block

Now comes the elegant part. The Megatron-LM paper showed that you can wire column-wise and row-wise splits together so that **only one AllReduce** is needed per MLP block.

Here is the trick:

1. **First linear layer** ($W_1$): split **column-wise**. Each GPU computes a subset of the intermediate activations.
2. **GeLU activation**: applied **locally** on each GPU. No communication needed! Why? Because each GPU has a complete slice of the intermediate features — GeLU is elementwise, so it does not need values from other GPUs.
3. **Second linear layer** ($W_2$): split **row-wise**. The input to this layer is already split across GPUs (from step 1), which is exactly what row-wise split expects.
4. **AllReduce (sum)**: combine the partial outputs.

The brilliant insight: the column-wise output (split features) naturally feeds into the row-wise input (split input). No gather or scatter is needed in between. The only communication is a single AllReduce at the very end.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_megatron_mlp_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class MegatronMLP:
    """
    Megatron-LM style tensor-parallel MLP block.

    Architecture:
        Input -> ColumnParallelLinear -> GeLU -> RowParallelLinear -> AllReduce -> Output

    Communication: Only ONE AllReduce per forward pass.
    """
    def __init__(self, W1, b1, W2, b2, world_size=2):
        self.world_size = world_size
        self.col_linear = ColumnParallelLinear(W1, b1, world_size)
        self.row_linear = RowParallelLinear(W2, b2, world_size)

        # Store full weights for reference computation
        self.W1, self.b1 = W1, b1
        self.W2, self.b2 = W2, b2

The single-GPU forward is straightforward: standard linear -> GeLU -> linear.

In [ ]:
def megatron_forward_single_gpu(mlp, x):
    """Non-parallel forward (ground truth)."""
    h = x @ mlp.W1 + mlp.b1
    h = F.gelu(h)
    return h @ mlp.W2 + mlp.b2

# Attach to the class
MegatronMLP.forward_single_gpu = lambda self, x: megatron_forward_single_gpu(self, x)

Now the tensor-parallel version. Each GPU computes its shard, applies GeLU locally, and we sum at the end.

In [ ]:
def megatron_forward_tp(mlp, x):
    """Tensor-parallel forward (simulated)."""
    # Step 1: Column-wise linear — each GPU gets a shard
    gpu_intermediates = []
    for rank in range(mlp.world_size):
        h_shard = mlp.col_linear.forward(x, rank)
        gpu_intermediates.append(h_shard)

    # Step 2: GeLU — applied LOCALLY, no communication!
    gpu_activations = [F.gelu(h) for h in gpu_intermediates]

    # Step 3: Row-wise linear — partial output per GPU
    partial_outputs = []
    for rank in range(mlp.world_size):
        partial = gpu_activations[rank] @ mlp.row_linear.shards[rank]
        partial_outputs.append(partial)

    # Step 4: AllReduce (sum) — the ONLY communication
    output = sum(partial_outputs)
    if mlp.row_linear.full_bias is not None:
        output = output + mlp.row_linear.full_bias
    return output

MegatronMLP.forward_tp = lambda self, x: megatron_forward_tp(self, x)

Let us test the Megatron MLP and verify it matches single-GPU computation.

In [ ]:
# Test the Megatron MLP
torch.manual_seed(42)
hidden_dim = 16
intermediate_dim = 32  # 2x expansion (real models use ~3.5x)
batch_size = 4

W1 = torch.randn(hidden_dim, intermediate_dim) * 0.1
b1 = torch.randn(intermediate_dim) * 0.01
W2 = torch.randn(intermediate_dim, hidden_dim) * 0.1
b2 = torch.randn(hidden_dim) * 0.01
X_mlp = torch.randn(batch_size, hidden_dim)

mlp = MegatronMLP(W1, b1, W2, b2, world_size=2)

Y_single = mlp.forward_single_gpu(X_mlp)
Y_tp = mlp.forward_tp(X_mlp)

print("Megatron-LM MLP Test")
print(f"  Input: ({batch_size}, {hidden_dim})")
print(f"  Intermediate: ({batch_size}, {intermediate_dim})")
print(f"  Output: {Y_tp.shape}")
print(f"  Matches single-GPU: {torch.allclose(Y_tp, Y_single, atol=1e-5)}")
print(f"  Max error: {(Y_tp - Y_single).abs().max().item():.2e}")
print(f"\n  Communication: 1 AllReduce of {Y_tp.numel()} elements")
print(f"  GeLU applied locally (no communication needed)")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_megatron_flow_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Megatron MLP Data Flow

Let us create a visual showing exactly what each GPU computes and where communication happens.

In [ ]:
# Set up the figure for the Megatron MLP flow diagram
fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Megatron-LM Tensor-Parallel MLP (2-way)',
             fontsize=16, fontweight='bold', pad=20)

# Colors
blue = '#3498db'
green = '#2ecc71'
orange = '#e67e22'
red = '#e74c3c'
gray = '#95a5a6'

# Input X — full on both GPUs
rect = mpatches.FancyBboxPatch((5, 9), 4, 0.6, boxstyle="round,pad=0.1",
                                facecolor=gray, edgecolor='black', linewidth=1.5)
ax.add_patch(rect)
ax.text(7, 9.3, 'Input X (full copy on both GPUs)',
        ha='center', va='center', fontsize=10, fontweight='bold')

Now we draw the column-wise linear layer stage.

In [ ]:
# Arrows from input to column-wise linear
ax.annotate('', xy=(5.5, 8.2), xytext=(5.5, 8.8),
            arrowprops=dict(arrowstyle='->', lw=2, color=blue))
ax.annotate('', xy=(8.5, 8.2), xytext=(8.5, 8.8),
            arrowprops=dict(arrowstyle='->', lw=2, color=green))

# Column-wise linear boxes
rect_g0 = mpatches.FancyBboxPatch((3.5, 7.4), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=blue, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g0)
ax.text(5, 7.7, 'GPU 0: X @ W1_col', ha='center', va='center',
        fontsize=9, fontweight='bold', color=blue)

rect_g1 = mpatches.FancyBboxPatch((7.5, 7.4), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=green, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g1)
ax.text(9, 7.7, 'GPU 1: X @ W2_col', ha='center', va='center',
        fontsize=9, fontweight='bold', color=green)
ax.text(12.5, 7.7, '<- Column Parallel', ha='center', va='center',
        fontsize=9, color='gray', style='italic')

Adding the GeLU activation stage — applied locally with no communication.

In [ ]:
# Arrows to GeLU
ax.annotate('', xy=(5, 6.6), xytext=(5, 7.2),
            arrowprops=dict(arrowstyle='->', lw=2, color=blue))
ax.annotate('', xy=(9, 6.6), xytext=(9, 7.2),
            arrowprops=dict(arrowstyle='->', lw=2, color=green))

# GeLU (local, no communication)
rect_g0 = mpatches.FancyBboxPatch((3.5, 5.9), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=blue, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g0)
ax.text(5, 6.2, 'GPU 0: GeLU', ha='center', va='center',
        fontsize=9, fontweight='bold', color=blue)

rect_g1 = mpatches.FancyBboxPatch((7.5, 5.9), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=green, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g1)
ax.text(9, 6.2, 'GPU 1: GeLU', ha='center', va='center',
        fontsize=9, fontweight='bold', color=green)
ax.text(12.5, 6.2, '<- No Communication!', ha='center', va='center',
        fontsize=9, color=red, fontweight='bold')

Now the row-wise linear stage and arrows leading to it.

In [ ]:
# Arrows to row-wise linear
ax.annotate('', xy=(5, 5.1), xytext=(5, 5.7),
            arrowprops=dict(arrowstyle='->', lw=2, color=blue))
ax.annotate('', xy=(9, 5.1), xytext=(9, 5.7),
            arrowprops=dict(arrowstyle='->', lw=2, color=green))

# Row-wise linear boxes
rect_g0 = mpatches.FancyBboxPatch((3.5, 4.3), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=blue, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g0)
ax.text(5, 4.6, 'GPU 0: h0 @ W1_row', ha='center', va='center',
        fontsize=9, fontweight='bold', color=blue)

rect_g1 = mpatches.FancyBboxPatch((7.5, 4.3), 3, 0.6, boxstyle="round,pad=0.1",
                                   facecolor=green, edgecolor='black', linewidth=1.5, alpha=0.3)
ax.add_patch(rect_g1)
ax.text(9, 4.6, 'GPU 1: h1 @ W2_row', ha='center', va='center',
        fontsize=9, fontweight='bold', color=green)
ax.text(12.5, 4.6, '<- Row Parallel', ha='center', va='center',
        fontsize=9, color='gray', style='italic')

Finally, the AllReduce and output — the only communication point in the entire MLP.

In [ ]:
# Arrows converging to AllReduce
ax.annotate('', xy=(7, 3.3), xytext=(5, 4.1),
            arrowprops=dict(arrowstyle='->', lw=2, color=blue))
ax.annotate('', xy=(7, 3.3), xytext=(9, 4.1),
            arrowprops=dict(arrowstyle='->', lw=2, color=green))

# AllReduce circle and output
circle = plt.Circle((7, 2.9), 0.5, facecolor=orange,
                     edgecolor='black', linewidth=2, alpha=0.5)
ax.add_patch(circle)
ax.text(7, 2.9, 'AllReduce\n(Sum)', ha='center', va='center',
        fontsize=8, fontweight='bold')
ax.text(12.5, 2.9, '<- ONLY communication!', ha='center', va='center',
        fontsize=9, color=red, fontweight='bold')

ax.annotate('', xy=(7, 1.6), xytext=(7, 2.3),
            arrowprops=dict(arrowstyle='->', lw=2, color='black'))

rect = mpatches.FancyBboxPatch((5, 1.0), 4, 0.6, boxstyle="round,pad=0.1",
                                facecolor=gray, edgecolor='black', linewidth=1.5)
ax.add_patch(rect)
ax.text(7, 1.3, 'Output Y', ha='center', va='center',
        fontsize=10, fontweight='bold')

plt.tight_layout()
plt.savefig('megatron_mlp_flow.png', dpi=100, bbox_inches='tight')
plt.show()
print("Notice: GeLU is applied locally. Only ONE AllReduce per MLP block!")

In [ ]:
#@title 🎧 Listen: Why Gelu Local
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_why_gelu_local.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🤔 Why Can GeLU Be Applied Locally?

This is a subtle but important point. After the column-wise split, GPU 0 has the activations corresponding to the **first half of intermediate features**, and GPU 1 has the **second half**. GeLU is an elementwise function — it operates on each element independently. It does not need to "see" the other GPU's features. So each GPU can safely apply GeLU to its local shard.

If instead we had used a non-elementwise operation here (like softmax across features), we would need communication. The choice of GeLU (or any elementwise activation) is precisely what makes this one-AllReduce design work.

In [ ]:
#@title 🎧 Listen: Tp Attention Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_tp_attention_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Tensor-Parallel Multi-Head Attention

For self-attention, the splitting strategy is even more natural. Attention heads are **independent** — each head computes its own Q, K, V projections and attention scores without needing information from other heads.

So we simply assign different heads to different GPUs:
- With 8 heads and 2 GPUs, GPU 0 handles heads 0-3, GPU 1 handles heads 4-7
- Each GPU computes full attention for its assigned heads
- The outputs are concatenated and projected back

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/15_tp_attention_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class TensorParallelMultiHeadAttention:
    """
    Tensor-parallel multi-head attention.

    Each GPU handles a subset of the attention heads.
    The Q, K, V projections are split column-wise (by heads).
    The output projection is split row-wise.
    """
    def __init__(self, hidden_dim, num_heads, world_size=2):
        assert num_heads % world_size == 0, \
            "num_heads must be divisible by world_size"

        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.world_size = world_size
        self.heads_per_gpu = num_heads // world_size

        # Full weight matrices (each GPU only stores its shard in practice)
        torch.manual_seed(123)
        self.W_q = torch.randn(hidden_dim, hidden_dim) * 0.02
        self.W_k = torch.randn(hidden_dim, hidden_dim) * 0.02
        self.W_v = torch.randn(hidden_dim, hidden_dim) * 0.02
        self.W_o = torch.randn(hidden_dim, hidden_dim) * 0.02

The single-GPU forward pass computes all heads together (ground truth).

In [ ]:
def tp_attention_forward_single(attn, x):
    """Non-parallel forward pass (ground truth)."""
    batch, seq_len, _ = x.shape
    d = attn.hidden_dim
    nh = attn.num_heads
    hd = attn.head_dim

    Q = (x @ attn.W_q).view(batch, seq_len, nh, hd).transpose(1, 2)
    K = (x @ attn.W_k).view(batch, seq_len, nh, hd).transpose(1, 2)
    V = (x @ attn.W_v).view(batch, seq_len, nh, hd).transpose(1, 2)

    scores = (Q @ K.transpose(-2, -1)) / (hd ** 0.5)
    attn_weights = F.softmax(scores, dim=-1)
    attn_out = (attn_weights @ V).transpose(1, 2).contiguous().view(batch, seq_len, d)

    return attn_out @ attn.W_o

The tensor-parallel version distributes heads across GPUs. Each GPU computes Q, K, V for its heads only.

In [ ]:
def tp_attention_forward_parallel(attn, x):
    """Tensor-parallel forward pass (simulated)."""
    batch, seq_len, _ = x.shape
    hd = attn.head_dim
    hpg = attn.heads_per_gpu
    local_dim = hpg * hd  # Dimension per GPU

    partial_outputs = []
    for rank in range(attn.world_size):
        start = rank * local_dim
        end = (rank + 1) * local_dim

        # Column-wise split of Q, K, V projections
        Q_local = (x @ attn.W_q[:, start:end]).view(
            batch, seq_len, hpg, hd).transpose(1, 2)
        K_local = (x @ attn.W_k[:, start:end]).view(
            batch, seq_len, hpg, hd).transpose(1, 2)
        V_local = (x @ attn.W_v[:, start:end]).view(
            batch, seq_len, hpg, hd).transpose(1, 2)

Each GPU runs attention locally and computes its partial output projection.

In [ ]:
        # Local attention (no communication!)
        scores = (Q_local @ K_local.transpose(-2, -1)) / (hd ** 0.5)
        attn_weights = F.softmax(scores, dim=-1)
        attn_out = (attn_weights @ V_local).transpose(1, 2).contiguous()
        attn_out = attn_out.view(batch, seq_len, local_dim)

        # Row-wise split of output projection
        partial = attn_out @ attn.W_o[start:end, :]
        partial_outputs.append(partial)

    # AllReduce (sum)
    return sum(partial_outputs)

Let us test that tensor-parallel attention matches the single-GPU version.

In [ ]:
# Test tensor-parallel attention
torch.manual_seed(42)
hidden_dim = 64
num_heads = 8
seq_len = 16
batch_size = 2

attn = TensorParallelMultiHeadAttention(hidden_dim, num_heads, world_size=2)
X_attn = torch.randn(batch_size, seq_len, hidden_dim)

Y_single = tp_attention_forward_single(attn, X_attn)
Y_tp = tp_attention_forward_parallel(attn, X_attn)

print("Tensor-Parallel Multi-Head Attention Test")
print(f"  Hidden dim: {hidden_dim}, Num heads: {num_heads}, Heads per GPU: {num_heads // 2}")
print(f"  Input: ({batch_size}, {seq_len}, {hidden_dim})")
print(f"  Output: {Y_tp.shape}")
print(f"  Matches single-GPU: {torch.allclose(Y_tp, Y_single, atol=1e-5)}")
print(f"  Max error: {(Y_tp - Y_single).abs().max().item():.2e}")
print(f"\n  Each GPU computes {num_heads // 2} heads independently")
print(f"  Communication: 1 AllReduce after output projection")

In [ ]:
#@title 🎧 Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/16_transition_to_todos.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. 🔧 Your Turn

### TODO 1: Implement Column-Wise Split and Verify

Your task: given a weight matrix and an input, implement the column-wise split manually and verify that concatenating the results matches the full computation.

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/17_todo1.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def column_wise_split_matmul(X, W, world_size):
    """
    Split the weight matrix W along columns and compute X @ W
    using simulated tensor parallelism.

    Args:
        X: Input tensor of shape (batch, in_features)
        W: Weight matrix of shape (in_features, out_features)
        world_size: Number of simulated GPUs

    Returns:
        Y: Output tensor of shape (batch, out_features)
    """
    out_features = W.shape[1]
    shard_size = out_features // world_size

    # ============ TODO ============
    # Step 1: Split W into `world_size` column shards
    # Step 2: Compute X @ W_shard for each shard
    # Step 3: Concatenate along the last dimension
    # ==============================

    gpu_outputs = []
    for rank in range(world_size):
        W_shard = ???  # YOUR CODE: extract columns
        Y_shard = ???  # YOUR CODE: compute the matmul
        gpu_outputs.append(Y_shard)

    Y = ???  # YOUR CODE: combine the outputs
    return Y

Run this cell to check your implementation of TODO 1.

In [ ]:
# Verification for TODO 1
torch.manual_seed(99)
X_todo = torch.randn(3, 8)
W_todo = torch.randn(8, 12)

Y_expected_todo = X_todo @ W_todo

for ws in [2, 3, 4]:
    Y_yours = column_wise_split_matmul(X_todo, W_todo, world_size=ws)
    match = torch.allclose(Y_yours, Y_expected_todo, atol=1e-6)
    print(f"  world_size={ws}: shape={Y_yours.shape}, correct={match}",
          "PASS" if match else "FAIL")

print("\nIf all show PASS, your implementation is correct!")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/18_todo1_followup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 🔧 TODO 2: Build a Complete Megatron-Style Transformer Block

Given `ColumnParallelLinear` and `RowParallelLinear` from above, assemble a complete tensor-parallel Transformer block that includes **both** MLP and attention. The block should follow the standard pre-norm Transformer architecture:

$$\text{Output} = x + \text{MLP}(\text{LayerNorm}(x + \text{Attention}(\text{LayerNorm}(x))))$$

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/19_todo2.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class TensorParallelTransformerBlock:
    """
    A single Transformer block with tensor-parallel MLP and attention.

    Architecture (pre-norm):
        x -> LayerNorm -> TP-Attention -> residual
          -> LayerNorm -> TP-MLP -> residual -> output
    """
    def __init__(self, hidden_dim, num_heads, intermediate_dim, world_size=2):
        self.world_size = world_size
        self.hidden_dim = hidden_dim

        # Layer norms (not parallelized — replicated on each GPU)
        self.ln1_weight = torch.ones(hidden_dim)
        self.ln1_bias = torch.zeros(hidden_dim)
        self.ln2_weight = torch.ones(hidden_dim)
        self.ln2_bias = torch.zeros(hidden_dim)

        # TP Attention
        self.attn = TensorParallelMultiHeadAttention(
            hidden_dim, num_heads, world_size)

        # TP MLP weights
        torch.manual_seed(456)
        W1 = torch.randn(hidden_dim, intermediate_dim) * 0.02
        b1 = torch.zeros(intermediate_dim)
        W2 = torch.randn(intermediate_dim, hidden_dim) * 0.02
        b2 = torch.zeros(hidden_dim)
        self.mlp = MegatronMLP(W1, b1, W2, b2, world_size)

Layer norm helper — replicated on every GPU, not parallelized.

In [ ]:
def block_layer_norm(x, weight, bias, eps=1e-5):
    """Standard layer norm (replicated, not parallelized)."""
    mean = x.mean(dim=-1, keepdim=True)
    var = x.var(dim=-1, keepdim=True, unbiased=False)
    return weight * (x - mean) / torch.sqrt(var + eps) + bias

TensorParallelTransformerBlock.layer_norm = staticmethod(block_layer_norm)

Now the tensor-parallel forward pass — fill in the TODO blanks.

In [ ]:
def block_forward_tp(block, x):
    """
    Tensor-parallel forward pass.
    Fill in the blanks below.
    """
    # ============ TODO ============
    # Step 1: Pre-norm + TP Attention + residual
    # Step 2: Pre-norm + TP MLP + residual
    # ==============================

    # Step 1
    h = block_layer_norm(x, block.ln1_weight, block.ln1_bias)
    attn_out = ???  # YOUR CODE: tensor-parallel attention
    x = ???         # YOUR CODE: residual connection

    # Step 2
    h = block_layer_norm(x, block.ln2_weight, block.ln2_bias)
    mlp_out = ???   # YOUR CODE: tensor-parallel MLP
    x = ???         # YOUR CODE: residual connection

    return x

TensorParallelTransformerBlock.forward_tp = lambda self, x: block_forward_tp(self, x)

And the single-GPU forward pass for comparison (ground truth).

In [ ]:
def block_forward_single(block, x):
    """Non-parallel forward (ground truth)."""
    h = block_layer_norm(x, block.ln1_weight, block.ln1_bias)
    attn_out = tp_attention_forward_single(block.attn, h)
    x = x + attn_out

    h = block_layer_norm(x, block.ln2_weight, block.ln2_bias)
    mlp_out = block.mlp.forward_single_gpu(h)
    x = x + mlp_out
    return x

TensorParallelTransformerBlock.forward_single = lambda self, x: block_forward_single(self, x)

Run this cell to verify your TODO 2 implementation.

In [ ]:
# Verification for TODO 2
torch.manual_seed(42)
block = TensorParallelTransformerBlock(
    hidden_dim=64, num_heads=8, intermediate_dim=128, world_size=2
)
X_block = torch.randn(2, 16, 64)

Y_single = block.forward_single(X_block)
Y_tp = block.forward_tp(X_block)

if Y_tp is not None:
    match = torch.allclose(Y_tp, Y_single, atol=1e-4)
    max_err = (Y_tp - Y_single).abs().max().item()
    print(f"Output shape: {Y_tp.shape}")
    print(f"Matches single-GPU: {match} {'PASS' if match else 'FAIL'}")
    print(f"Max error: {max_err:.2e}")

    if match:
        print("\nYour tensor-parallel Transformer block works correctly!")
        print("Communication per block: 2 AllReduces (1 for attention, 1 for MLP)")
else:
    print("forward_tp returned None. Did you fill in the TODO?")

In [ ]:
#@title 🎧 Before You Start
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/20_todo2_followup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Communication Analysis

Let us analyze the communication cost of tensor parallelism and understand **why it must stay within a single node**.

In [ ]:
#@title 🎧 Listen: Communication Analysis
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/21_communication_analysis.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def compute_tp_communication(hidden_dim, seq_len, batch_size,
                             num_layers, world_size):
    """
    Compute the AllReduce communication volume for TP.

    Each Transformer layer has 2 AllReduce operations:
    - 1 for attention output projection
    - 1 for MLP second linear layer
    """
    bytes_per_element = 2  # fp16
    tensor_elements = batch_size * seq_len * hidden_dim
    tensor_bytes = tensor_elements * bytes_per_element

    # Ring AllReduce: 2 * (N-1)/N * tensor_size
    allreduce_bytes = 2 * (world_size - 1) / world_size * tensor_bytes
    per_layer_bytes = 2 * allreduce_bytes
    total_bytes = per_layer_bytes * num_layers

    return {
        'per_allreduce_bytes': allreduce_bytes,
        'per_layer_bytes': per_layer_bytes,
        'total_bytes': total_bytes,
        'total_gb': total_bytes / 1e9,
    }

Now let us compute the actual numbers for a Llama 2 70B-scale model.

In [ ]:
# Llama 2 70B parameters
hidden_dim = 8192
seq_len = 4096
batch_size = 1
num_layers = 80

print("Communication volume for Llama 2 70B-scale model:")
print(f"  hidden_dim={hidden_dim}, seq_len={seq_len}, layers={num_layers}")
print("-" * 60)

for ws in [2, 4, 8]:
    stats = compute_tp_communication(hidden_dim, seq_len, batch_size, num_layers, ws)
    print(f"\n  TP world_size = {ws}:")
    print(f"    Per AllReduce: {stats['per_allreduce_bytes']/1e6:.1f} MB")
    print(f"    Per layer: {stats['per_layer_bytes']/1e6:.1f} MB")
    print(f"    Total (all layers): {stats['total_gb']:.2f} GB")

In [ ]:
#@title 🎧 Listen: Critical Point
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/22_critical_point.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Critical Point**: This communication happens inside **every single layer**, for **every single micro-batch**. It is not like Data Parallelism where you communicate once per training step — tensor parallelism communicates constantly.

This is why interconnect bandwidth is the bottleneck.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/23_nvlink_vs_infiniband.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Bandwidth comparison: NVLink vs InfiniBand
nvlink_bw = 900    # GB/s (H100 NVSwitch)
ib_bw = 50         # GB/s (400 Gb/s InfiniBand, realistic)

# Time per training step (all layers, one micro-batch)
stats_8way = compute_tp_communication(8192, 4096, 1, 80, 8)
total_gb = stats_8way['total_gb']

time_nvlink = total_gb / nvlink_bw * 1000  # ms
time_ib = total_gb / ib_bw * 1000          # ms

print(f"Total communication per micro-batch (TP=8): {total_gb:.2f} GB")
print(f"\n  On NVLink (900 GB/s):     {time_nvlink:.2f} ms  -- Fast enough")
print(f"  On InfiniBand (50 GB/s):  {time_ib:.2f} ms  -- {time_ib/time_nvlink:.0f}x slower!")
print(f"\nThis is why TP is ALWAYS within a single node (NVLink).")
print(f"Cross-node TP would be {time_ib/time_nvlink:.0f}x slower -- a dealbreaker.")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/24_communication_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Communication Latency vs TP Degree

In [ ]:
# Left panel: Communication volume
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

world_sizes = [1, 2, 4, 8]
volumes = []
for ws in world_sizes:
    if ws == 1:
        volumes.append(0)
    else:
        stats = compute_tp_communication(8192, 4096, 1, 80, ws)
        volumes.append(stats['total_gb'])

bars = axes[0].bar(range(len(world_sizes)), volumes,
                    color=['#95a5a6', '#3498db', '#2ecc71', '#e67e22'],
                    edgecolor='black', linewidth=1.2)
axes[0].set_xticks(range(len(world_sizes)))
axes[0].set_xticklabels([f'TP={ws}' for ws in world_sizes])
axes[0].set_ylabel('Communication Volume (GB)', fontsize=11)
axes[0].set_title('Communication per Micro-Batch\n(Llama 70B-scale)',
                   fontsize=12, fontweight='bold')
for bar, vol in zip(bars, volumes):
    if vol > 0:
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                     f'{vol:.1f} GB', ha='center', fontsize=10)

Now the right panel comparing NVLink vs InfiniBand latency.

In [ ]:
# Right panel: Latency comparison — compute the times
nvlink_times = []
ib_times = []
for ws in world_sizes:
    if ws == 1:
        nvlink_times.append(0)
        ib_times.append(0)
    else:
        stats = compute_tp_communication(8192, 4096, 1, 80, ws)
        nvlink_times.append(stats['total_gb'] / nvlink_bw * 1000)
        ib_times.append(stats['total_gb'] / ib_bw * 1000)

x = np.arange(len(world_sizes))
width = 0.35
axes[1].bar(x - width/2, nvlink_times, width, label='NVLink (900 GB/s)',
            color='#2ecc71', edgecolor='black', linewidth=1.2)
axes[1].bar(x + width/2, ib_times, width, label='InfiniBand (50 GB/s)',
            color='#e74c3c', edgecolor='black', linewidth=1.2)

Format the latency comparison axes and render the full figure.

In [ ]:
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'TP={ws}' for ws in world_sizes])
axes[1].set_ylabel('Latency (ms)', fontsize=11)
axes[1].set_title('Communication Latency per Micro-Batch\n(NVLink vs InfiniBand)',
                   fontsize=12, fontweight='bold')
axes[1].legend(fontsize=10)
axes[1].set_yscale('log')
axes[1].set_ylim(bottom=0.01)

plt.tight_layout()
plt.savefig('tp_communication_analysis.png', dpi=100, bbox_inches='tight')
plt.show()
print("NVLink is ~18x faster than InfiniBand -- that is why TP stays within one node.")

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/25_full_transformer.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. 🎯 Full Tensor-Parallel Transformer

Let us build a multi-layer Transformer, run both single-GPU and tensor-parallel forward passes, and verify they produce identical results.

In [ ]:
class TensorParallelTransformer:
    """
    A multi-layer Transformer with tensor parallelism.

    Simulates a real TP setup where:
    - Q, K, V projections are split column-wise (by heads)
    - MLP first linear is column-wise, second is row-wise
    - Only 2 AllReduces per layer
    """
    def __init__(self, num_layers, hidden_dim, num_heads,
                 intermediate_dim, world_size=2):
        self.num_layers = num_layers
        self.world_size = world_size
        self.blocks = []

        for i in range(num_layers):
            torch.manual_seed(i * 1000)
            block = TensorParallelTransformerBlock(
                hidden_dim, num_heads, intermediate_dim, world_size
            )
            self.blocks.append(block)

Add forward methods for both single-GPU and tensor-parallel execution.

In [ ]:
def transformer_forward_single(model, x):
    """Non-parallel forward (ground truth)."""
    for block in model.blocks:
        x = block.forward_single(x)
    return x

def transformer_forward_tp(model, x):
    """Tensor-parallel forward."""
    for block in model.blocks:
        x = block.forward_tp(x)
    return x

TensorParallelTransformer.forward_single = lambda self, x: transformer_forward_single(self, x)
TensorParallelTransformer.forward_tp = lambda self, x: transformer_forward_tp(self, x)

Build and test the full transformer.

In [ ]:
# Build and test the full transformer
torch.manual_seed(42)
num_layers = 4
hidden_dim = 512
num_heads = 8
intermediate_dim = 1024  # 2x expansion
world_size = 2
seq_len = 32
batch_size = 2

print("Building Tensor-Parallel Transformer...")
print(f"  Layers: {num_layers}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  Attention heads: {num_heads} ({num_heads // world_size} per GPU)")
print(f"  Intermediate dim: {intermediate_dim}")
print(f"  TP world_size: {world_size}")
print(f"  Input: ({batch_size}, {seq_len}, {hidden_dim})")
print()

model = TensorParallelTransformer(
    num_layers, hidden_dim, num_heads, intermediate_dim, world_size)
X_final = torch.randn(batch_size, seq_len, hidden_dim)

Now run both forward passes and compare.

In [ ]:
# Forward passes
Y_single = model.forward_single(X_final)
Y_tp = model.forward_tp(X_final)

match = torch.allclose(Y_tp, Y_single, atol=1e-3)
max_err = (Y_tp - Y_single).abs().max().item()
mean_err = (Y_tp - Y_single).abs().mean().item()

print(f"Output shape: {Y_tp.shape}")
print(f"Matches single-GPU: {match}")
print(f"Max absolute error: {max_err:.2e}")
print(f"Mean absolute error: {mean_err:.2e}")

Let us look at the detailed communication breakdown.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/26_communication_breakdown.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Communication breakdown
total_allreduces = num_layers * 2  # 2 per layer (attention + MLP)
elements_per_allreduce = batch_size * seq_len * hidden_dim
bytes_per_allreduce = elements_per_allreduce * 2  # fp16

print("=" * 60)
print("  COMMUNICATION VOLUME BREAKDOWN")
print("=" * 60)
print(f"  AllReduces per layer:     2 (attention + MLP)")
print(f"  Total AllReduces:         {total_allreduces}")
print(f"  Elements per AllReduce:   {elements_per_allreduce:,}")
print(f"  Bytes per AllReduce:      {bytes_per_allreduce:,} ({bytes_per_allreduce/1024:.1f} KB)")
print(f"  Total communication:      {total_allreduces * bytes_per_allreduce:,} bytes")
print(f"                            ({total_allreduces * bytes_per_allreduce / 1e6:.2f} MB)")
print("=" * 60)
print()
print(f"  Memory saved per GPU:     ~{1 - 1/world_size:.0%} of weight memory")
print(f"  Compute per GPU:          ~{1/world_size:.0%} of FLOPs")

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/27_error_memory_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 📊 Visualization: Error and Memory Analysis

In [ ]:
# Output comparison heatmap and per-layer error
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1. Absolute error map
diff = (Y_tp - Y_single).abs()
im0 = axes[0].imshow(diff[0].detach().numpy(), cmap='hot', aspect='auto')
axes[0].set_title(f'Absolute Error Map\n(batch=0, max={max_err:.1e})',
                   fontsize=11, fontweight='bold')
axes[0].set_xlabel('Hidden dimension')
axes[0].set_ylabel('Sequence position')
plt.colorbar(im0, ax=axes[0], shrink=0.8)

# 2. Per-layer error accumulation
layer_errors = []
x_single = X_final.clone()
x_tp = X_final.clone()
for i, blk in enumerate(model.blocks):
    x_single = blk.forward_single(x_single)
    x_tp = blk.forward_tp(x_tp)
    layer_errors.append((x_tp - x_single).abs().max().item())

axes[1].plot(range(1, num_layers + 1), layer_errors, 'o-',
             color='#e74c3c', linewidth=2, markersize=8)
axes[1].set_xlabel('Layer', fontsize=11)
axes[1].set_ylabel('Max Absolute Error', fontsize=11)
axes[1].set_title('Error Accumulation Across Layers',
                   fontsize=11, fontweight='bold')
axes[1].set_xticks(range(1, num_layers + 1))
axes[1].grid(True, alpha=0.3)

And the memory savings panel.

In [ ]:
# 3. Memory savings bar chart
tp_degrees = [1, 2, 4, 8]
total_params = num_layers * (
    3 * hidden_dim * hidden_dim +     # Q, K, V projections
    hidden_dim * hidden_dim +          # Output projection
    hidden_dim * intermediate_dim +    # MLP W1
    intermediate_dim * hidden_dim      # MLP W2
)
mem_per_gpu = [total_params * 2 / tp / 1e6 for tp in tp_degrees]

bars = axes[2].bar(range(len(tp_degrees)), mem_per_gpu,
                    color=['#e74c3c', '#3498db', '#2ecc71', '#f39c12'],
                    edgecolor='black', linewidth=1.2)
axes[2].set_xticks(range(len(tp_degrees)))
axes[2].set_xticklabels([f'TP={tp}' for tp in tp_degrees])
axes[2].set_ylabel('Weight Memory per GPU (MB)', fontsize=11)
axes[2].set_title(f'Memory Savings\n({num_layers}-layer model)',
                   fontsize=11, fontweight='bold')
for bar, mem in zip(bars, mem_per_gpu):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                 f'{mem:.1f} MB', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('tensor_parallelism_final.png', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
#@title 🎧 Transition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/28_unified_demo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Putting It All Together

Now that we have built every component individually, let us combine them into a unified end-to-end demo. We will create a single input, run it through tensor-parallel attention, a tensor-parallel MLP, and the full Transformer — then compare all outputs side by side.

In [ ]:
# Unified end-to-end demo: all TP components in one pipeline
torch.manual_seed(2024)

# Shared configuration
demo_hidden = 64
demo_heads = 8
demo_intermediate = 128
demo_seq = 16
demo_batch = 2
demo_ws = 2

x_demo = torch.randn(demo_batch, demo_seq, demo_hidden)

print("=" * 60)
print("  UNIFIED TENSOR PARALLELISM DEMO")
print("=" * 60)
print(f"  Input shape: ({demo_batch}, {demo_seq}, {demo_hidden})")
print(f"  World size (simulated GPUs): {demo_ws}")
print()

First, run the standalone tensor-parallel attention module.

In [ ]:
#@title 🎧 Code Walkthrough
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/29_unified_demo_run.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# --- Component 1: TP Attention ---
attn_demo = TensorParallelMultiHeadAttention(demo_hidden, demo_heads, demo_ws)
y_attn_single = tp_attention_forward_single(attn_demo, x_demo)
y_attn_tp = tp_attention_forward_parallel(attn_demo, x_demo)
attn_match = torch.allclose(y_attn_tp, y_attn_single, atol=1e-5)
attn_err = (y_attn_tp - y_attn_single).abs().max().item()

print("Component 1: TP Multi-Head Attention")
print(f"  Heads total: {demo_heads}, Heads per GPU: {demo_heads // demo_ws}")
print(f"  Output shape: {y_attn_tp.shape}")
print(f"  Single-GPU match: {attn_match}  (max error: {attn_err:.2e})")
print()

Next, the Megatron-style tensor-parallel MLP.

In [ ]:
# --- Component 2: TP MLP ---
torch.manual_seed(2024)
W1_demo = torch.randn(demo_hidden, demo_intermediate) * 0.02
b1_demo = torch.zeros(demo_intermediate)
W2_demo = torch.randn(demo_intermediate, demo_hidden) * 0.02
b2_demo = torch.zeros(demo_hidden)

mlp_demo = MegatronMLP(W1_demo, b1_demo, W2_demo, b2_demo, demo_ws)

# Use a 2D input for the MLP (flatten seq into batch)
x_mlp_demo = x_demo.view(-1, demo_hidden)
y_mlp_single = mlp_demo.forward_single_gpu(x_mlp_demo)
y_mlp_tp = mlp_demo.forward_tp(x_mlp_demo)
mlp_match = torch.allclose(y_mlp_tp, y_mlp_single, atol=1e-5)
mlp_err = (y_mlp_tp - y_mlp_single).abs().max().item()

print("Component 2: Megatron TP MLP")
print(f"  Intermediate dim: {demo_intermediate}")
print(f"  Output shape: {y_mlp_tp.shape}")
print(f"  Single-GPU match: {mlp_match}  (max error: {mlp_err:.2e})")
print()

Finally, the full multi-layer Transformer combining both components.

In [ ]:
# --- Component 3: Full TP Transformer ---
demo_layers = 3
model_demo = TensorParallelTransformer(
    demo_layers, demo_hidden, demo_heads, demo_intermediate, demo_ws)

y_model_single = model_demo.forward_single(x_demo)
y_model_tp = model_demo.forward_tp(x_demo)
model_match = torch.allclose(y_model_tp, y_model_single, atol=1e-3)
model_err = (y_model_tp - y_model_single).abs().max().item()

print(f"Component 3: Full TP Transformer ({demo_layers} layers)")
print(f"  Output shape: {y_model_tp.shape}")
print(f"  Single-GPU match: {model_match}  (max error: {model_err:.2e})")
print(f"  AllReduces per forward: {demo_layers * 2}")
print()

Summary table of the unified demo results.

In [ ]:
# Summary table
print("=" * 60)
print("  SUMMARY: ALL COMPONENTS VERIFIED")
print("=" * 60)
results = [
    ("TP Attention", attn_match, attn_err, 1),
    ("TP MLP (Megatron)", mlp_match, mlp_err, 1),
    (f"TP Transformer ({demo_layers}L)", model_match, model_err, demo_layers * 2),
]
print(f"  {'Component':<25} {'Match':<8} {'Max Error':<12} {'AllReduces'}")
print(f"  {'-'*25} {'-'*8} {'-'*12} {'-'*10}")
for name, m, e, ar in results:
    status = "PASS" if m else "FAIL"
    print(f"  {name:<25} {status:<8} {e:<12.2e} {ar}")

print()
print("  Every component produces identical results to single-GPU computation.")
print("  The ONLY overhead is the AllReduce communication at known points.")

Let us visualize the output distributions for all three components side by side.

In [ ]:
#@title 🎧 What to Look For
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/30_unified_demo_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualization: output distributions for all 3 components
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, y_s, y_t) in zip(axes, [
    ("TP Attention", y_attn_single, y_attn_tp),
    ("TP MLP", y_mlp_single.view(demo_batch, demo_seq, demo_hidden),
               y_mlp_tp.view(demo_batch, demo_seq, demo_hidden)),
    (f"TP Transformer ({demo_layers}L)", y_model_single, y_model_tp),
]):
    vals_s = y_s.detach().flatten().numpy()
    vals_t = y_t.detach().flatten().numpy()
    ax.hist(vals_s, bins=50, alpha=0.5, label='Single GPU', color='#3498db')
    ax.hist(vals_t, bins=50, alpha=0.5, label='Tensor Parallel', color='#e74c3c')
    ax.set_title(name, fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)
    ax.set_xlabel('Activation value')
    ax.set_ylabel('Count')

plt.suptitle('Output Distributions: Single-GPU vs Tensor-Parallel',
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('putting_it_all_together.png', dpi=100, bbox_inches='tight')
plt.show()
print("The distributions overlap perfectly — TP is mathematically exact.")

In [ ]:
#@title 🎧 Listen: Celebration
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/31_celebration.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
print("=" * 60)
print("  You Have Implemented Tensor Parallelism from Scratch!")
print("=" * 60)
print()
print("  What you built:")
print("  - Column-wise parallel linear layer")
print("  - Row-wise parallel linear layer")
print("  - Megatron-LM style tensor-parallel MLP")
print("  - Tensor-parallel multi-head attention")
print("  - Full tensor-parallel Transformer (4 layers)")
print("  - Verified correctness against non-parallel computation")
print("  - Analyzed communication costs")
print("  - Unified demo combining all components")
print()
print("  Key takeaways:")
print("  - TP splits individual weight matrices across GPUs")
print("  - Column split -> concatenate; Row split -> AllReduce (sum)")
print("  - Megatron-LM: column-wise first, GeLU locally, row-wise second")
print("  - Only 2 AllReduces per Transformer layer")
print("  - TP MUST use NVLink (within a single node)")
print()
print("  Next up: Pipeline Parallelism -- splitting by model DEPTH!")

In [ ]:
#@title 🎧 Wrap-Up
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/32_reflection_and_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### 🤔 Reflection Questions

1. **Why does Megatron-LM use column-wise split for the first MLP layer and row-wise for the second?**
   Think about the shapes. The column-wise output naturally splits the intermediate features across GPUs. The row-wise input expects its input to be split across GPUs. These dovetail perfectly — no communication needed between them (except the elementwise GeLU).

2. **Why can GeLU be applied locally without communication?**
   GeLU is elementwise — it operates on each value independently. After the column-wise split, each GPU has a complete (though smaller) set of features. Applying GeLU to these local features is exactly the same as applying it to the corresponding entries in the full tensor.

In [ ]:
# Quick experiment: verify that GeLU commutes with column splitting
# This cements the intuition from Reflection Question 2
torch.manual_seed(42)
x_verify = torch.randn(4, 12)
full_gelu = F.gelu(x_verify)
split_gelu = torch.cat([F.gelu(x_verify[:, :6]), F.gelu(x_verify[:, 6:])], dim=1)
print(f"GeLU(full) == cat(GeLU(split_0), GeLU(split_1)): {torch.allclose(full_gelu, split_gelu)}")
print("Elementwise operations always commute with column splitting.")

3. **Why is TP limited to within a single node in practice?**
   TP communicates inside every single layer (2 AllReduces per layer). For a 70B model with 80 layers, that is 160 AllReduces per micro-batch. This communication must be fast enough to not bottleneck the computation. NVLink provides 900 GB/s, while cross-node InfiniBand is ~18x slower at ~50 GB/s.

### 🏆 Optional Challenges

1. **Vary the TP degree**: Modify the code to test TP with `world_size=4` and `world_size=8`. Verify correctness still holds. How does the error change?

2. **fp16 precision**: Convert the weights and inputs to `torch.float16` and re-run. Does the error increase significantly? This is a real concern in production systems.

3. **Asymmetric split**: What if the weight matrix dimensions are not evenly divisible by `world_size`? Implement a version that handles padding or uneven shards.

4. **Compute-to-communication ratio**: For a given `hidden_dim` and `seq_len`, calculate the ratio of FLOPs (compute) to bytes transferred (communication). Plot this ratio as `hidden_dim` grows. Why does TP become more efficient for larger models?